## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_df = pd.read_csv("../Weather_Database1/WeatherPy_database.csv")
weather_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,50.49,98,100,6.89,overcast clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,60.01,64,77,9.66,broken clouds
2,2,Poum,NC,-20.2333,164.0167,76.53,74,81,10.87,broken clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,75.34,73,5,5.99,clear sky
4,4,Dikson,RU,73.5069,80.5464,-4.40,100,98,6.15,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the weather_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Poum,NC,-20.2333,164.0167,76.53,74,81,10.87,broken clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,75.34,73,5,5.99,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,75.07,74,100,4.43,light rain
8,8,Butaritari,KI,3.0707,172.7902,81.59,74,50,16.28,scattered clouds
10,10,Kapaa,US,22.0752,-159.3190,75.18,87,52,3.00,broken clouds
13,13,San Andres,CO,12.5847,-81.7006,80.31,77,86,8.88,overcast clouds
16,16,Avarua,CK,-21.2078,-159.7750,77.05,94,20,8.05,light rain
17,17,Kloulklubed,PW,7.0419,134.2556,75.70,72,87,5.75,overcast clouds
18,18,Presidente Medici,BR,-11.1753,-61.9014,83.80,75,72,4.50,broken clouds
20,20,Victoria,HK,22.2855,114.1577,78.96,89,11,4.00,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                223
City                   223
Country                221
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
clean_df.count()

City_ID                221
City                   221
Country                221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Poum,NC,76.53,broken clouds,-20.2333,164.0167,
3,Sao Filipe,CV,75.34,clear sky,14.8961,-24.4956,
7,Rikitea,PF,75.07,light rain,-23.1203,-134.9692,
8,Butaritari,KI,81.59,scattered clouds,3.0707,172.7902,
10,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,
13,San Andres,CO,80.31,overcast clouds,12.5847,-81.7006,
16,Avarua,CK,77.05,light rain,-21.2078,-159.7750,
17,Kloulklubed,PW,75.70,overcast clouds,7.0419,134.2556,
18,Presidente Medici,BR,83.80,broken clouds,-11.1753,-61.9014,
20,Victoria,HK,78.96,few clouds,22.2855,114.1577,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   221
Country                221
Max Temp               221
Current Description    221
Lat                    221
Lng                    221
Hotel Name             221
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))